# EDA

In [14]:
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 3.1 MB/s eta 0:00:00 0:00:01


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [66]:
df = pd.read_csv("../artifacts/data_ingestion/lapse-rate.csv")

In [67]:
df

,CHANNEL1,CHANNEL2,CHANNEL3,ENTRY AGE,SEX,POLICY TYPE 1,POLICY TYPE 2,POLICY TYPE 3,PAYMENT MODE,POLICY STATUS,BENEFIT,NON LAPSE GUARANTEED,SUBSTANDARD RISK,NUMBER OF ADVANCE PREMIUM,INITIAL BENEFIT,Full Benefit?,Policy Year (Decimal),Policy Year,Premium,Issue Date
0,1,1,1,36,F,1,1,A,Annually,Inforce,200000.0,NO NLG,0.0,0,0.0,N,8.000000,9,280.0,11-Aug
1,1,1,1,42,M,1,2,A,Annually,Inforce,100000.0,NO NLG,0.0,0,0.0,N,8.000000,9,200.0,11-Aug
2,1,1,2,40,M,2,3,A,Annually,Lapse,80000.0,NO NLG,0.0,0,0.0,N,8.000000,9,1289.0,11-Aug
3,1,1,3,39,M,1,2,A,Monthly,Lapse,100000.0,NO NLG,0.0,0,0.0,N,8.000000,9,216.0,11-Aug
4,1,1,4,44,M,1,2,A,Annually,Lapse,100000.0,NO NLG,0.0,0,0.0,N,7.916667,8,140.0,11-Sep
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185555,8,3,0,31,F,12,58,A,Annually,Inforce,247158.0,NO NLG,0.0,0,0.0,N,0.000000,1,5000.0,19-Aug
185556,8,3,0,36,M,12,58,A,Annually,Inforce,200000.0,NO NLG,0.0,0,0.0,N,0.000000,1,4734.0,19-Aug
185557,8,3,0,24,M,8,52,C,Quaterly,Inforce,17000.0,NO NLG,0.0,0,0.0,N,0.000000,1,NaN,19-Aug
185558,8,3,0,35,F,12,58,A,Annually,Inforce,100000.0,NO NLG,0.0,0,0.0,N,0.000000,1,2023.0,19-Aug


In [68]:
df['Issue Date'].value_counts()

Issue Date
16-Dec    4884
16-Jun    4319
15-Dec    3739
16-Nov    3731
15-Jun    3417
          ... 
11-Nov      86
11-Oct      82
11-Sep      79
11-Aug      76
12-Jan      70
Name: count, Length: 97, dtype: int64

In [69]:
df.dtypes.unique()

array([dtype('int64'), dtype('O'), dtype('float64')], dtype=object)

In [81]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest, chi2

# Assuming 'df' is your original DataFrame and 'target' is your target variable
# If you don't have a target variable, you'll need to specify one

# 1. Select columns with object dtype
object_columns = df.select_dtypes(include=['object']).columns

df_non_categorical = df.drop(columns=object_columns)

# 2. Apply OrdinalEncoder to object columns
encoder = OrdinalEncoder()
df_encoded = pd.DataFrame(encoder.fit_transform(df[object_columns]), 
                          columns=object_columns, 
                          index=df.index)

In [82]:
df_encoded

,SEX,POLICY TYPE 3,PAYMENT MODE,POLICY STATUS,NON LAPSE GUARANTEED,Full Benefit?,Issue Date
0,0.0,0.0,0.0,2.0,3.0,0.0,0.0
1,1.0,0.0,0.0,2.0,3.0,0.0,0.0
2,1.0,0.0,0.0,3.0,3.0,0.0,0.0
3,1.0,0.0,1.0,3.0,3.0,0.0,0.0
4,1.0,0.0,0.0,3.0,3.0,0.0,4.0
...,...,...,...,...,...,...,...
185555,0.0,0.0,0.0,2.0,3.0,0.0,90.0
185556,1.0,0.0,0.0,2.0,3.0,0.0,90.0
185557,1.0,2.0,2.0,2.0,3.0,0.0,90.0
185558,0.0,0.0,0.0,2.0,3.0,0.0,90.0


In [83]:
# 3. Apply chi-squared test
# Assuming 'target' is your target variable. Replace it with your actual target column name
target = df['POLICY STATUS']  # Replace 'target' with your actual target column name
selector = SelectKBest(chi2, k='all')  # Select all features initially
selector.fit(df_encoded, target)

# Get feature scores and p-values
scores = pd.DataFrame({
    'feature': object_columns,
    'score': selector.scores_,
    'p_value': selector.pvalues_
})

# Sort features by score in descending order
scores = scores.sort_values('score', ascending=False)

In [84]:
print(scores)

                feature          score       p_value
6            Issue Date  212279.617087  0.000000e+00
3         POLICY STATUS   28820.519036  0.000000e+00
1         POLICY TYPE 3    4095.632968  0.000000e+00
2          PAYMENT MODE     403.080673  6.006970e-86
5         Full Benefit?     122.214141  1.797535e-25
0                   SEX      33.356863  1.009469e-06
4  NON LAPSE GUARANTEED       4.959311  2.914980e-01


In [85]:
# 4. Select top features (e.g., top 5 or based on p-value)
top_features = scores.head(5)['feature'].tolist()  # Select top 5 features
# Alternatively, you can select based on p-value:
# top_features = scores[scores['p_value'] < 0.05]['feature'].tolist()


In [86]:
print(top_features)

['Issue Date', 'POLICY STATUS', 'POLICY TYPE 3', 'PAYMENT MODE', 'Full Benefit?']


In [87]:
# 5. Create DataFrame with selected features
df_selected = df_encoded[top_features]

In [88]:
df_selected

,Issue Date,POLICY STATUS,POLICY TYPE 3,PAYMENT MODE,Full Benefit?
0,0.0,2.0,0.0,0.0,0.0
1,0.0,2.0,0.0,0.0,0.0
2,0.0,3.0,0.0,0.0,0.0
3,0.0,3.0,0.0,1.0,0.0
4,4.0,3.0,0.0,0.0,0.0
...,...,...,...,...,...
185555,90.0,2.0,0.0,0.0,0.0
185556,90.0,2.0,0.0,0.0,0.0
185557,90.0,2.0,2.0,2.0,0.0
185558,90.0,2.0,0.0,0.0,0.0


In [89]:

# 7. Combine remaining original DataFrame with selected encoded features
df_final = pd.concat([df_non_categorical, df_selected], axis=1)

In [90]:
# Print results
print("Original DataFrame shape:", df.shape)
print("Final DataFrame shape:", df_final.shape)
print("Selected features:", top_features)
print("\nFeature scores:")
print(scores)


Original DataFrame shape: (185560, 20)
Final DataFrame shape: (185560, 18)
Selected features: ['Issue Date', 'POLICY STATUS', 'POLICY TYPE 3', 'PAYMENT MODE', 'Full Benefit?']

Feature scores:
                feature          score       p_value
6            Issue Date  212279.617087  0.000000e+00
3         POLICY STATUS   28820.519036  0.000000e+00
1         POLICY TYPE 3    4095.632968  0.000000e+00
2          PAYMENT MODE     403.080673  6.006970e-86
5         Full Benefit?     122.214141  1.797535e-25
0                   SEX      33.356863  1.009469e-06
4  NON LAPSE GUARANTEED       4.959311  2.914980e-01


In [91]:
df_final.head()

,CHANNEL1,CHANNEL2,CHANNEL3,ENTRY AGE,POLICY TYPE 1,POLICY TYPE 2,BENEFIT,SUBSTANDARD RISK,NUMBER OF ADVANCE PREMIUM,INITIAL BENEFIT,Policy Year (Decimal),Policy Year,Premium,Issue Date,POLICY STATUS,POLICY TYPE 3,PAYMENT MODE,Full Benefit?
0,1,1,1,36,1,1,200000.0,0.0,0,0.0,8.000000,9,280.0,0.0,2.0,0.0,0.0,0.0
1,1,1,1,42,1,2,100000.0,0.0,0,0.0,8.000000,9,200.0,0.0,2.0,0.0,0.0,0.0
2,1,1,2,40,2,3,80000.0,0.0,0,0.0,8.000000,9,1289.0,0.0,3.0,0.0,0.0,0.0
3,1,1,3,39,1,2,100000.0,0.0,0,0.0,8.000000,9,216.0,0.0,3.0,0.0,1.0,0.0
4,1,1,4,44,1,2,100000.0,0.0,0,0.0,7.916667,8,140.0,4.0,3.0,0.0,0.0,0.0


In [92]:
target

0         Inforce
1         Inforce
2           Lapse
3           Lapse
4           Lapse
           ...   
185555    Inforce
185556    Inforce
185557    Inforce
185558    Inforce
185559    Inforce
Name: POLICY STATUS, Length: 185560, dtype: object